# **Questions**

**Will the AND-1 riboswitch cleave itself when both of its OBS are bound?**

Yes, that should happen when you bind both OBS.

**Will the OR-1 riboswitch cleave itself when neither of its OBS are bound?** 

No, the OR switch should need at least one OBS bound to activate. 

**What behavior do we expect from the YES-1 riboswitch?**

The YES-1 gate will be inactive until it has its OBS bound. 

In [167]:
#import relevant packages

%matplotlib inline
import sys
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from Bio import Entrez
from Bio import SeqFeature
from Bio import SeqIO

import subprocess

from IPython.display import IFrame, HTML, Image, display

import sqlite3
conn = sqlite3.connect('my.db')
c = conn.cursor()

# Part 1: Obtain Sequences and Build a Data Table

<img src="Screen Shot 2019-10-02 at 4.06.58 PM.png">
Figure 2a: YES-1 riboswitch: 

In [168]:
#YES-1 
two_a = ['two_a','GGGCGACCCUGAUGAGCUUGAGUUUAGCUCGUCACUGUCCAGGUUCAAUCAGGCGAAACGGUGAAAGCCGUAGGUUGCCC']

#NOT-1 
four_a = ['four_a','GGCAGGUACAUACAGCUGAUGAGUCCCAAAUAGGACGAAACGCGACACACACCACUAAACCGUGCAGUGUUUUGCGUCCUGUAUUCCACUGC']

#AND-1 
five_a = ['five_a','GGGCGACCCUGAUGGAGCUUGGUUUAGUAUUUACAGCUCCAUACAUGAGGUGUUAUCCCUAUGCAAGUUCAUCAGGCGAAACGGUGAAAGCCGUAGGUUGCCCAGAGACAAU']

#OR-1 
six_a = ['six_a','GGGCGACCCUGAUGAGCUUGGUUGAGUAUUUACAGCUCCAUACAUGAGGUGUUCUCCCUACGCAAGUUCGAUCAGGCGAAACGGUGAAAGCCGUAGGUUGCCC']


In [169]:
#store the name and start/stop locations of blue/red regions for each riboswitch
yes1 = ["YES-1", [26, 47], [0, 0], [16,21], [49,54]]
not1 = ["NOT-1", [44, 66], [0, 0], [40,43], [74,77]]
and1 = ["AND-1", [30,45], [49, 64], [16, 23], [70, 77]]
or1 = ["OR-1", [27, 46], [47, 66], [16, 26], [67, 77]]

ribos = [yes1, not1, and1, or1]

In [170]:
#delete and recreate riboswitch table
c.execute("""DROP TABLE riboswitch""")
c.execute("""CREATE TABLE riboswitch (name TEXT,
                                blue1start INT,
                                blue1end INT,
                                blue2start INT,
                                blue2end INT,
                                red1start INT,
                                red1end INT,
                                red2start INT,
                                red2end INT);""")

OperationalError: database is locked

In [ ]:
#populate the riboswitch table

for ribo in range(len(ribos)):
    current = ribos[ribo]
    #pull data from riboswitch lists
    ribo_name = current[0]
    blue1start = current[1][0]
    blue1end = current[1][1]
    blue2start = current[2][0]
    blue2end = current[2][1]
    red1start = current[3][0]
    red1end = current[3][1]
    red2start = current[4][0]
    red2end = current[4][1]
    params = (ribo_name,blue1start,blue1end, blue2start,blue2end, red1start, red1end, red2start,red2end)
    c.execute("""INSERT INTO riboswitch 
                         VALUES(?,?,?,?,?,?,?,?,?);""", params)
    conn.commit()


In [171]:
#test if table works 
c.execute("""SELECT * FROM riboswitch WHERE name = "YES-1" """)

# Part 2: Use RNA_Fold to Develop Folded RNA Structures

In [172]:
#RNA Fold subprocess command

def RNA_Fold(sequence, constraint_sequence = 0, version_number = '_'):
  
    #checks to see if you're inputting a constrained sequence or not
    if constraint_sequence != 0:
        p=subprocess.run(['RNAfold', '--constraint'],
                    input=bytes(sequence[1] + '\n' + constraint_sequence,'ascii'),
                    stdout=subprocess.PIPE,
                    stderr=subprocess.PIPE)
        #changes the file name so that you can tell apart the different OBS bind sites
        file_name = 'convert rna.ps ' + sequence[0] + '_constraint' + version_number + '.png'
    else:
        p=subprocess.run(['RNAfold'],
                    input=bytes(sequence[1],'ascii'),
                    stdout=subprocess.PIPE,
                    stderr=subprocess.PIPE)
        file_name = 'convert rna.ps ' + sequence[0] + '.png'
    print(p.stderr.decode())
    subprocess.run(['RNAplot','-o','svg'],
                  input=bytes(p.stdout.decode(),'ascii'),
                  stderr=subprocess.PIPE)
    subprocess.call(file_name, shell=True)

    return

In [173]:
RNA_Fold(two_a)
RNA_Fold(four_a)
RNA_Fold(five_a)
RNA_Fold(six_a)

# Unconstrained Structures

# **YES-1**

![](two_a.png)

two_a: This structure is not the same as the one in the paper. It has more stem loops than the conformation described in the paper. One of the original stem loops from the paper is split into two stems in the RNA_Fold conformation. Also, the red regions don't align so the ribozyme is inactive. 

# **NOT-1**

![](four_a.png)

four_a: This structure looks similar to that of the paper. The number of stem loops is the same and the general struture matches that of the paper. This makes sense because the unbound conformation should be the most stable, and the RNA_Fold predicts structures primarily based on having a low free energy. The red regions are paired in this structure, making it active. 

# **AND-1**

![](five_a.png)

five_a: This structure is very close to the one depicted in the paper. It has the same number of stem loops along with the same pairing patterns, and the only differences are the angles at which the stem loops branch off (probably due to software differences in plotting). The red regions don't bp with each other so its inactive. 

# **OR-1**

![](six_a.png)

six_a: This structure is very similar to that of the paper. The stem loops and bubbles are all the same. One of the two red regions is bound, which makes sense for an 'or' gate. This ribozyme should still be inactive though. 

# Part 3: Constrained Structures

In [174]:
#function that creates the sequence with "." and "x" to represent the constraint locations

def generate_constraints(sequence, start, stop):
    file = ''
    for i in range(0, start):
        file += '.'
    for i in range(start, stop):
        file += 'x'
    for i in range(stop, len(sequence)):
        file += '.'
    return file

In [175]:
#creates the constraint files and stores them as strings

yes_constraint = generate_constraints(two_a[1], 26, 47)
no_constraint = generate_constraints(four_a[1], 44, 66)
and_constraint1 = generate_constraints(five_a[1],30,45)
and_constraint2 = generate_constraints(five_a[1], 49, 64)
or_constraint1 = generate_constraints(six_a[1], 27, 46) 
or_constraint2 = generate_constraints(six_a[1], 47, 66)

In [176]:
#outputs the constraint strings for cross checking

print(yes_constraint)
print(no_constraint)
print(and_constraint1)
print(and_constraint2)
print(or_constraint1)
print(or_constraint2)
and_constraint3 = '..............................xxxxxxxxxxxxxxx....xxxxxxxxxxxxxxx................................................'
or_constraint3 =  '...........................xxxxxxxxxxxxxxxxxxx.xxxxxxxxxxxxxxxxxxx.....................................'

..........................xxxxxxxxxxxxxxxxxxxxx.................................
............................................xxxxxxxxxxxxxxxxxxxxxx..........................
..............................xxxxxxxxxxxxxxx...................................................................
.................................................xxxxxxxxxxxxxxx................................................
...........................xxxxxxxxxxxxxxxxxxx.........................................................
...............................................xxxxxxxxxxxxxxxxxxx.....................................


In [177]:
#creates a new .png file with the structure of the constrained RNA fold

RNA_Fold(two_a, yes_constraint)
RNA_Fold(four_a, no_constraint)
RNA_Fold(five_a, and_constraint1, 'one')
RNA_Fold(five_a, and_constraint2, 'two')
RNA_Fold(five_a, and_constraint3, 'three')
RNA_Fold(six_a, or_constraint1, 'one')
RNA_Fold(six_a, or_constraint2, 'two')
RNA_Fold(six_a, or_constraint3, 'three')

# Part 4: Constrained Structures + Analysis

![](two_a_constraint_.png)

two_a_constraint: The folding structure matches that of the paper. Also, the red regions line up, meaning that its an active ribozyme. 

![](four_a_constraint_.png)

four_a_constraint: The folding structure matches that of the paper. Also, the red regions do not line up, meaning that its an inactive ribozyme. 

![](five_a_constraintone.png)

five_a_constraintone: (OBS1) The folding structure does not match the paper. The red regions don't align so its inactive. 

![](five_a_constrainttwo.png)

five_a_constrainttwo: (OBS2) The folding structure does not match the paper. The red regions don't align so its inactive. 

![](five_a_constraintthree.png)

five_a_constraintthree: (OBS1 and OBS2) The folding structure does not match the paper. The red regions don't align so its inactive. 

![](six_a_constraintone.png)

six_a_constraintone: (OBS1) The folding structure does not match the paper. The red regions don't align so its inactive, even though its supposed to be active. 

![](six_a_constrainttwo.png)

six_a_constrainttwo: (OBS2) The folding structure matches the paper. The red regions align so its active, which makes sense because one OBS is bound. 

![](six_a_constraintthree.png)

six_a_constraintthree: (OBS1 and OBS2) The folding structure matches the paper. Moreover, the structure is the exact same as constraint two. The red regions align so its active, which makes sense because both OBS are bound. 

# Part 5: Truth Table: Effectiveness of Ribozyme Switches

In [178]:
d = {'OBS1': ['T', 'F', 'T', 'F'], 'OBS2': ['T', 'F', 'F', 'T'], 'AND-1 active': ['no', 'no', 'no', 'no'], 'OR-1 active': ['yes', 'no', 'no', 'yes']}
df = pd.DataFrame(data=d)
df

,OBS1,OBS2,AND-1 active,OR-1 active
0,T,T,no,yes
1,F,F,no,no
2,T,F,no,no
3,F,T,no,yes


The AND-1 and OR-1 riboswitches do not work as the paper claims. The AND-1 switch doesn't even activate when both OBS are bound, making it immediately nonfunctional. The OR-1 switch doesn't work fully, but does activate when OBS-2 is bound. It doesn't work as an OR gate though because OBS1 fails to activate it. 